# 네이버 영화평 감성 분석

In [1]:
!pip install Konlpy > /dev/null

In [2]:
import numpy as np
import pandas as pd
import warnings 
warnings.filterwarnings("ignore")

In [3]:
# 네이버 영화 리뷰 데이터로 검색

In [4]:
train_df = pd.read_csv("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", sep = "\t")
test_df = pd.read_csv("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt", sep = "\t")

In [5]:
display(train_df.head(5))
display(test_df.head(5))


,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0


### 1. 데이터 전처리
  - train 데이터셋

In [6]:
# Null 데이터가 있는지 확인
train_df.isna().sum()

id          0
document    5
label       0
dtype: int64

In [7]:
# null 데이터 제거
train_df.dropna(how = "any", inplace = True)
train_df.shape

(149995, 3)

In [8]:
# 중복여부 확인
train_df.document.nunique()

146182

In [9]:
# 중복 데이터 제거
train_df.drop_duplicates(subset = ["document"], inplace = True)
train_df.shape

(146182, 3)

In [10]:
# 긍정 (1)/ 부정(0)
train_df.label.value_counts(), np.unique(train_df.label.values, return_counts = True)

(0    73342
 1    72840
 Name: label, dtype: int64, (array([0, 1]), array([73342, 72840])))

- 테스트 데이터셋

In [11]:
test_df.isnull().sum()

id          0
document    3
label       0
dtype: int64

In [12]:
test_df.dropna(how = "any", inplace = True)

In [13]:
test_df.document.nunique()

49157

In [14]:
test_df.drop_duplicates(subset = ["document"], inplace = True)
test_df.shape

(49157, 3)

In [15]:
test_df.label.value_counts()
np.unique(test_df.label, return_counts = True)

(array([0, 1]), array([24446, 24711]))

### 2. 텍스트 전처리

In [16]:
# 한글 이외의 문자는 공백으로 처리하고 strip
train_df.document = train_df.document.str.replace("[^ㄱ-하-ㅣ가-힣]"," ").str.strip()

In [17]:
# ""만 남은 데이터 --> np.nan으로 대체한 후 제거
train_df.document.replace("",np.nan,inplace = True)
train_df.document.isna().sum()

747

In [18]:
train_df.dropna(how = "any", inplace = True)
train_df.shape

(145435, 3)

- 테스트 데이터셋

In [19]:
test_df.document = test_df.document.str.replace("[^ㄱ-하-ㅣ가-힣]"," ").str.strip()
test_df.document.replace("",np.nan,inplace = True)
test_df.dropna(how = "any", inplace = True)
test_df.shape

(48860, 3)

- 전처리가 끝난 데이터는 저장

In [20]:
train_df.to_csv("naver_movie_train_전처리완료.csv", sep = "\t", index = False)
test_df.to_csv("naver_movie_test_전처리완료.csv", sep = "\t", index = False)


### 3.한글 처리

In [ ]:
from konlpy.tag import Okt
okt = Okt()


In [ ]:
text = "교도소 이야기구만 솔직히 재미는 없다"
okt.morphs(text)

['교도소', '이야기', '구만', '솔직히', '재미', '는', '없다']

In [ ]:
okt.morphs(text, stem = True)

['교도소', '이야기', '구만', '솔직하다', '재미', '는', '없다']

In [ ]:
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를',
             '으로','자','에','와','한','하다','을','ㅋㅋ','ㅠㅠ','ㅎㅎ']
# " ".join([word for word in okt.morphs(text, stem = True) if word not in stopwords])

In [ ]:
from tqdm.notebook import tqdm

X_train = []
for sentence in tqdm(train_df.document):
    morphs = okt.morphs(sentence, stem = True)
    tmp_str = " ".join([word for word in morphs if word not in stopwords])
    X_train.append(tmp_str)
X_train

In [ ]:
from tqdm.notebook import tqdm

X_test = []
for sentence in tqdm(test_df.document):
    morphs = okt.morphs(sentence, stem = True)
    tmp_str = " ".join([word for word in morphs if word not in stopwords])
    X_test.append(tmp_str)
X_test

In [ ]:
X_train = train_df.document.values
X_test = test_df.document.values
y_train = train_df.label.values
y_test = test_df.label.values

### 4. 피쳐 변환, 모델 학습

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
cvector = CountVectorizer()
lr = LogisticRegression(random_state = 2022)
pipeline = Pipeline([
        ("CVECT", cvector), ("LR", lr)
])

In [ ]:
%time pipeline.fit(X_train,y_train )

CPU times: user 13.3 s, sys: 10.5 s, total: 23.8 s
Wall time: 13.8 s


Pipeline(steps=[('CVECT', CountVectorizer()),
                ('LR', LogisticRegression(random_state=2022))])

### 5. 실제 데이터 테스트

In [ ]:
review1 = """이게 몰입이ㅜ안됨 스토리가 연속적이지 않고 이솝우화처럼 스토리 여러개로 분할되서 
            에피소드가 ㅈㄴ여러개임근데 서로 관련성은 전혀 없음ㄹㅇ 보다가 도중에 나온거 처음 """
review2 = """중간중간 코믹요소가 들어가서 너무 재밌었고 음악이, 구둣발 소리가, 제 심장을 떨리게 만들었습니다.
             정말 시간가는줄 모르고 봤구요. 엔딩이 슬프지만... 해피엔딩을 바라는건 우리의 바람일뿐이고
              어쩌면 그게 현실인것 같은.. 정말 정말 재밌게 봤습니다.. 이걸 2022년 이제야 보다니... 한번 더 보고싶네요.
            맥주 홀짝거리며 한번 더 봐야겠습니다. 도경수 배우님은 아이돌이라고는 생각도 못했는데.. 
            놀랍네요. 덕분에 도경수님 그리고 엑소 팬이 되었습니다. 흥하세요"""


In [ ]:
# 전처리
import re
review1 = re.sub("[^가-힣]", " ", review1)
review2 = re.sub("[^가-힣]", " ", review2)

In [ ]:
morphs = okt.morphs(review1, stem = True)
review1 = " ".join([word for word in morphs if word not in stopwords])
morphs = okt.morphs(review2, stem = True)
review2 = " ".join([word for word in morphs if word not in stopwords])

In [ ]:
pipeline.predict([review1,review2])

array([0, 1])

### 6. 최적 파라메터 찾기


In [ ]:
from sklearn.model_selection import GridSearchCV
params = {
    "CVECT__ngram_range" : [(1,1),(1,2)],
    "CVECT__max_df" : [0.95,0.98],
    "LR__C" : [1,5,10]
}
grid_pipe = GridSearchCV(pipeline, params, cv = 3, scoring = "accuracy", n_jobs = -1)
%time grid_pipe.fit(X_train,y_train)

CPU times: user 43.5 s, sys: 22.2 s, total: 1min 5s
Wall time: 7min 24s


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('CVECT', CountVectorizer()),
                                       ('LR',
                                        LogisticRegression(random_state=2022))]),
             n_jobs=-1,
             param_grid={'CVECT__max_df': [0.95, 0.98],
                         'CVECT__ngram_range': [(1, 1), (1, 2)],
                         'LR__C': [1, 5, 10]},
             scoring='accuracy')

In [ ]:
grid_pipe.best_params_

{'CVECT__max_df': 0.95, 'CVECT__ngram_range': (1, 2), 'LR__C': 5}

In [ ]:
grid_pipe.best_estimator_.score(X_test,y_test)

0.8153704461727385